# Plot LRG chains
This jupyter notebook contains some code to make plots of the bestfits and contours for the abacus cubic LRG fits to both the mock mean and individual realisations.

In [3]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%pip install chainconsumer                  # Currently needed as not yet default in cosmodesi
from chainconsumer import ChainConsumer     

sys.path.append("../../../Barry/")     # Change this so that it points to where you have Barry installed

from barry.samplers import DynestySampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction
from barry.utils import weighted_avg_and_cov

# Read in the fitter class to get all the info on the fit
pfn = "./plots/desi_kp4_abacus_cubic_LRG/output/desi_kp4_abacus_cubic_LRG.fitter.pkl"
with open(pfn, 'rb') as pickle_file:
    fitter = pickle.load(pickle_file)

This code segment reads in the chains, plots the bestfit model vs. data and prepares stuff for contour plots and summary files

In [4]:
logging.info("Creating plots")

fitname = []
datanames = ["Xi", "Pk", "Pk_CV"]
c = [ChainConsumer(), ChainConsumer(), ChainConsumer(), ChainConsumer(), ChainConsumer(), ChainConsumer()]

# Loop over all the chains
stats = {}
output = {}
for posterior, weight, chain, evidence, model, data, extra in fitter.load():

    # Get the realisation number and redshift bin
    recon_bin = 0 if "Prerecon" in extra["name"] else 1
    data_bin = 0 if "Xi" in extra["name"] else 1 if "CV" not in extra["name"] else 2
    realisation = str(extra["name"].split()[-1]) if "realisation" in extra["name"] else "mean"
    chain_bin = int(2.0 * data_bin + recon_bin)
    print(extra["name"], recon_bin, data_bin, realisation)
    
    # Store the chain in a dictionary with parameter names
    df = pd.DataFrame(chain, columns=model.get_labels())

    # Compute alpha_par and alpha_perp for each point in the chain
    alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
    df["$\\alpha_\\parallel$"] = alpha_par
    df["$\\alpha_\\perp$"] = alpha_perp

    # Get the MAP point and set the model up at this point
    model.set_data(data)
    r_s = model.camb.get_data()["r_s"]
    max_post = posterior.argmax()
    params = df.loc[max_post]
    params_dict = model.get_param_dict(chain[max_post])
    for name, val in params_dict.items():
        model.set_default(name, val)

    # Get some useful properties of the fit, and plot the MAP model against the data if it's the mock mean or realisation 10 (chosen randomly!)
    new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=True)

    # Add the chain or MAP to the Chainconsumer plots
    extra.pop("realisation", None)
    if realisation == "mean":
        fitname.append(data[0]["name"].replace(" ", "_"))
        stats[fitname[recon_bin]] = []
        output[fitname[recon_bin]] = []
        c[chain_bin].add_chain(df, weights=weight, **extra, plot_contour=True, plot_point=False, show_as_1d_prior=False)
    else:
        c[chain_bin].add_marker(params, **extra)

    # Compute some summary statistics and add them to a dictionary
    mean, cov = weighted_avg_and_cov(
        df[
            [
                "$\\alpha_\\parallel$",
                "$\\alpha_\\perp$",
                "$\\Sigma_{nl,||}$",
                "$\\Sigma_{nl,\\perp}$",
            ]
        ],
        weight,
        axis=0,
    )

    corr = cov[1, 0] / np.sqrt(cov[0, 0] * cov[1, 1])
    stats[fitname[recon_bin]].append(
        [mean[0], mean[1], np.sqrt(cov[0, 0]), np.sqrt(cov[1, 1]), corr, new_chi_squared, mean[2], mean[3]]
    )
    output[fitname[recon_bin]].append(
        f"{realisation:s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {mean[3]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[1, 1]):6.4f}, {corr:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
    )

UnboundLocalError: local variable 'mi' referenced before assignment

Plot the contour plots and output the summary statistics

In [ ]:
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}
for recon_bin in range(len(c)):
    c[recon_bin].plotter.plot(
        truth=truth,
        parameters=["$\\alpha_\\parallel$", "$\\alpha_\\perp$"],
        legend=False,
        display=True,
    )

    # Save all the numbers to a file
    with open(dir_name + "/Barry_fit_" + fitname[recon_bin] + ".txt", "w") as f:
        f.write(
            "# Realisation, alpha_par, alpha_perp, Sigma_nl_par, Sigma_nl_perp, sigma_alpha_par, sigma_alpha_perp, corr_alpha_par_perp, rd_of_template, bf_chi2, dof\n"
        )
        for l in output[fitname[recon_bin]]:
            f.write(l + "\n")

        # And now the average of all the individual realisations
        f.write("# ---------------------------------------------------\n")
        f.write(
            "# <alpha_par>, <alpha_perp>, <Sigma_nl_par>, <Sigma_nl_perp>, <sigma_alpha_par>, <sigma_alpha_perp>, <corr_alpha_par_perp>, std_alpha_par, std_alpha_perp, corr_alpha_par_perp, <bf_chi2>\n"
        )
        f.write(
            f"{means[0]:6.4f}, {means[1]:6.4f}, {means[6]:6.4f}, {means[7]:6.4f}, {means[2]:6.4f}, {means[3]:6.4f}, {means[4]:6.4f}, {np.sqrt(covs[0, 0]):6.4f}, {np.sqrt(covs[1, 1]):6.4f}, {corr:6.4f}, {means[5]:7.3f}\n"
        )